<a href="https://colab.research.google.com/github/gleiciny/myapp/blob/main/coletar_dados_bitcoin_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:

import requests
import pandas as pd

def get_historical_data(coin_id, vs_currency, days):
    url = f'https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart'
    params = {
        'vs_currency': vs_currency,
        'days': days,
    }
    response = requests.get(url, params=params)
    data = response.json()
    prices = data['prices']
    df = pd.DataFrame(prices, columns=['timestamp', 'price'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df

# Coletar dados históricos de preço do Bitcoin nos últimos 365 dias
df = get_historical_data('bitcoin', 'usd', '365')
df.to_csv('bitcoin_historical_data.csv', index=False)
print('Dados históricos salvos em bitcoin_historical_data.csv')


Dados históricos salvos em bitcoin_historical_data.csv


In [27]:
import requests
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

def get_historical_data(coin_id, vs_currency, days):
    url = f'https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart'
    params = {
        'vs_currency': vs_currency,
        'days': days,
    }
    response = requests.get(url, params=params)
    data = response.json()
    prices = data['prices']
    df = pd.DataFrame(prices, columns=['timestamp', 'price'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df

# Coletar dados históricos de preço do Bitcoin nos últimos 365 dias
df = get_historical_data('bitcoin', 'usd', '365')
df['price_diff'] = df['price'].diff()  # Diferença de preço
df['price_pct_change'] = df['price'].pct_change()  # Mudança percentual do preço
df['price_7day_ma'] = df['price'].rolling(window=7).mean()  # Média móvel de 7 dias
df['price_30day_ma'] = df['price'].rolling(window=30).mean()  # Média móvel de 30 dias
df = df.dropna()
df.to_csv('bitcoin_historical_data.csv', index=False)

# Carregar dados históricos
df = pd.read_csv('bitcoin_historical_data.csv')

# Criar variáveis de entrada e saída
df['target'] = (df['price'].shift(-1) > df['price']).astype(int)
df = df.dropna()
X = df[['price', 'price_diff', 'price_pct_change', 'price_7day_ma', 'price_30day_ma']]
y = df['target']

# Normalizar os dados
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajuste de hiperparâmetros para o modelo Random Forest
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42), param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_

# Fazer previsões e calcular a acurácia
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia do modelo: {accuracy:.2f}')

# Visualizar a importância de cada variável
importances = best_model.feature_importances_
features = df[['price', 'price_diff', 'price_pct_change', 'price_7day_ma', 'price_30day_ma']].columns
importance_df = pd.DataFrame({'feature': features, 'importance': importances})
print(importance_df)


Fitting 3 folds for each of 108 candidates, totalling 324 fits
Acurácia do modelo: 0.46
            feature  importance
0             price    0.214310
1        price_diff    0.193714
2  price_pct_change    0.163017
3     price_7day_ma    0.195987
4    price_30day_ma    0.232972
